# Dreambooth fine-tuning for Stable Diffusion

This notebook shows how to "teach" Stable Diffusion a new concept via Dreambooth using 🤗 Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers).

We use the training script provided by huggingface [here](https://github.com/huggingface/diffusers/tree/main/examples/dreambooth) 

## Setup

Before running the script, we need to install the dependencies. Installing the dependencies with poetry caused some issues, so it is recommended to use a python virtual env manager to install dependencies.

In [1]:
# Clone the huggingface diffusers repo
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install -e .
%cd ..

fatal: destination path 'diffusers' already exists and is not an empty directory.
/Users/jonas/workspace/adomvi/adomvi/diffusers
Obtaining file:///Users/jonas/workspace/adomvi/adomvi/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.19.0.dev0-0.editable-py3-none-any.whl size=10590 sha256=ec059bcedf7640064097579c81498be54525eaa9cf1835db18ef71f91d4f3f6e
  Stored in directory: /private/var/folders/y5/kwb3b8ys5876gzp88vykkqjr0000gn/T/pip-ephem-wheel-cache-sog4eub_/wheels/f8/dc/3a/7fdc30db0235a22053dcb38120bbfb877212e56609af19dbc3
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.19.0.dev0
    Uninstalling diffusers-0.19.0.dev0:
      Successfully uninstalled

Install the dependencies required to run the training script

In [2]:
!pip install -r diffusers/examples/dreambooth/requirements.txt


[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# install missing dep bitsandbytes
!pip install bitsandbytes


[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


### initialize an 🤗Accelerate environment

In [4]:
from accelerate.utils import write_basic_config
write_basic_config()

/Users/jonas/workspace/adomvi/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jonas/workspace/adomvi/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Configuration already exists at /Users/jonas/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


False

## Download instance dataset

download a few images of a dog with

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir,
    repo_type="dataset",
    ignore_patterns=".gitattributes",
)

## Fine-tune the model

In [5]:
!export MODEL_NAME="CompVis/stable-diffusion-v1-4"
!export INSTANCE_DIR="./dog"
!export CLASS_DIR="dogs"
!export OUTPUT_DIR="dreambooth"

!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --train_text_encoder \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of sks dog" \
  --class_prompt="a photo of dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --use_8bit_adam \
  --gradient_checkpointing \
  --learning_rate=2e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=800

## Run inference

In [ ]:
from diffusers import DiffusionPipeline
import torch

model_id = "dreambooth"
pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

prompt = "A photo of sks dog in a bucket"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save("dog-bucket.png")